# load libraries and data

In [1]:
from torch import nn
import torch
from torch import tensor 
from torch.autograd import Variable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from sklearn.metrics import roc_auc_score
""" stratified k fold"""
from sklearn.model_selection import StratifiedKFold
import torch
import itertools
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm.notebook import tqdm
from pathlib import Path
import os

# psychosocial + rsfMRI + strurctural MRI + diffusion MRI

dataset_name = '.morct.rmvol.some'
train_out = Path(os.getcwd()+'/sex_classify/train'+dataset_name+'.csv')
test_out =Path(os.getcwd()+'/sex_classify/test'+dataset_name+'.csv')


train_data= pd.read_csv(train_out)
test_data= pd.read_csv(test_out)


In [2]:
# drop null columns 
train_data = train_data.drop(columns='fsqc_qc.y')
test_data=test_data.drop(columns='fsqc_qc.y')

In [3]:
target ='sex'
unused_feat = ['abcd_site','kfold' ,'race.ethnicity', 'subjectkey']

# Find the index of the column where the MRI begins.
start_mor_index = np.where(test_data.columns.values == "lh_bankssts_area._.1")[0][0]

start_con_index = np.where(test_data.columns.values == "con_L.BSTS_L.CACG_count")[0][0]

mor = list(test_data.columns[start_mor_index:start_con_index])
con = list(test_data.columns[start_con_index:])

# 5 flods CV 
Num_FOLDS  = 5

# the number of feature that you want to show 
Num_feat = 20

#Check the length of features 
print(len(mor))
print(len(con))


999
3486


# For getting feature importance 

In [4]:
def feature(Num_feat, clf_res, test_data_processed, features):
    # 5 Cross-Validation 
    importance_res = []
    for i in clf_res:
        importance_clf =i.feature_importances_
        importance_res.append(importance_clf)
    
    importance=[importance_res[0][i]/5+importance_res[1][i]/5+ importance_res[2][i]/5+ importance_res[3][i]/5+ importance_res[4][i]/5 for i in range(len(importance_res[1]))]

    feat_name_sort = test_data_processed[features].columns
    important_features = pd.DataFrame([importance],columns = feat_name_sort, index=['Importance']) 
    important_features =important_features.transpose().sort_values(by=['Importance'], ascending=False)
    important_features = important_features.head(50)
    return important_features

# Define function for preprocessing for Cross validation

In [5]:
def preprocessing (train_data, test_data, NUM_FOLDS):
    test_data_processed= test_data.fillna(0).reset_index(drop=True)
    train_data_processed = train_data.fillna(0).reset_index(drop=True)
    
    # __init__
    test_data_processed["kfold"] = -1
    train_data_processed["kfold"] = -1

    # frac: Decide how many percent of all rows to return -> frac=1 to return all data
    # random_state: To reproduce the same sampling in the future.
    # Sample: Select any sample from the data -> frac=1, only the order of the entire data is arbitrarily changed.
    train_data_processed = train_data_processed.sample(frac=1,random_state=2020).reset_index(drop=True)
    
    kf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state =0)
    
    for fold, (trn_, val_) in enumerate(kf.split(X=train_data_processed, y=train_data_processed[target])):
        # The column 'kfold' is a fold order designated when cross validation is performed.
        train_data_processed.loc[val_, 'kfold'] = fold
    
    print("done preprocessing")
    return train_data_processed, test_data_processed

# Finding best parameters

In [15]:
# Augmented
import torch
import itertools
from sklearn.metrics import confusion_matrix
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm.notebook import tqdm


def find_bestpar(fold, train_data_processed, test_data_processed, features):
    
    """Generate test data"""
    X_test = test_data_processed[features].values
    Y_test = test_data_processed[target].values
    
    # Store maximum auc
    max_auc= 0
    # Store maximum hypterparameter set
    max_hy = []
    """
    # define hyperparameter space : learning rate, 
    # Orginal hyperparameter space 
    n_ = [4,8,16]                              # 
    lr_ = [2e-2, 1e-2, 5e-3, 2e-3, 1e-3, 1e-4] # learning rate
    w_ = [0.01, 0.001, 0.0001]                 # weight decay
    g_ = [0.95, 0.99, 0.9]                     # scheduler params - gamma
    ss_ = [10, 20, 30]                         # scheduler params - step_size
    """
    # define hyperparameter space (quick version)
    n_ = [4,16]
    lr_ = [2e-2,1e-3]
    w_ = [0.01]
    g_ = [0.95,0.99]
    ss_ = [10,30]
    
    all_ = [n_, lr_, w_, g_, ss_]
    h_space = [s for s in itertools.product(*all_)]
    
    for hy in tqdm(h_space):
        """===================Cross Validation==================="""
        """validation & test result"""
        valid_res = []
        test_auc_res = []
        test_acc_res = []
     
        for i in range(1,fold):
        
    
            clf = TabNetClassifier(n_a = hy[0],
                                n_d = hy[0],
                                optimizer_params = dict(lr=hy[1], weight_decay=hy[2]),
                                scheduler_params={"step_size":hy[4], "gamma":hy[3]},
                                scheduler_fn=torch.optim.lr_scheduler.StepLR,
                                verbose=0)
            
            df_train = train_data_processed[train_data_processed['abcd_site'] != i]  # Four out of five are assigned for train data. 
            df_valid = train_data_processed[train_data_processed['abcd_site'] == i]  
            
            X_train = df_train[features].values
            Y_train = df_train[target].values
            
            X_valid = df_valid[features].values
            Y_valid = df_valid[target].values
  
            clf.fit(X_train, Y_train, eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
                    eval_name=['train', 'valid'], eval_metric=['auc'],
                    max_epochs=200 , patience=10)
       
            preds_acc = clf.predict(X_test)
            preds_prob = clf.predict_proba(X_test)
            test_auc = roc_auc_score(y_score=preds_prob[:,1], y_true=Y_test)
            test_acc = accuracy_score(preds_acc, Y_test)
            
            valid_res.append(clf.best_cost)
            test_auc_res.append(test_auc)
            test_acc_res.append(test_acc)
            print('[%3d/%4d] '%(i+1, fold),'Valid score: %2f'% clf.best_cost, 'Test AUC: %.3f%%'%test_auc, 'Test ACC: %.3f%%'%test_acc)
    
        """"print mean of valid and test"""
        print("===== valid, test score for each parameters =====")
        print("Validation mean: %3f"%np.mean(valid_res), "Test AUC mean: %3f"%np.mean(test_auc_res), "Test ACC mean: %3f"%np.mean(test_acc_res))

        if np.mean(test_auc_res)>max_auc:
            print("Find new maximum AUC!!")
            max_hy = hy
            max_auc = np.mean(test_auc_res)
    
    return max_hy


# Train with best parameter

In [16]:
def bestpar_tuning(fold, train_data_processed, test_data_processed, max_hy, features):
    hy = max_hy
    print("Max hy:" ,hy)
    X_test = test_data_processed[features].values
    Y_test = test_data_processed[target].values
    """result of validation & test"""
    valid_res = []
    test_auc_res = []
    test_acc_res = []
    clf_res = []
    preds_prob_res = []
    
    y_valid_true = []
    y_valid_pred = []
    y_test_pred = []
    
    y_valid_subject = []
    y_test_subject = []
    
    
    for i in range(1,fold):
        clf = TabNetClassifier(n_a = hy[0],n_d = hy[0],
                           optimizer_params = dict(lr=hy[1], weight_decay=hy[2]),
                           scheduler_params={"step_size":hy[4], "gamma":hy[3]},
                           scheduler_fn=torch.optim.lr_scheduler.StepLR,
                           verbose=0)
        
     
        df_train = train_data_processed[train_data_processed['abcd_site'] != i] 
        df_valid = train_data_processed[train_data_processed['abcd_site'] == i]  
        X_train = df_train[features].values
        Y_train = df_train[target].values
            
        X_valid = df_valid[features].values
        Y_valid = df_valid[target].values
        
    
        # Bring subject key before learning 
        y_valid_subject.append(df_valid['subjectkey'].values)
        y_test_subject.append(test_data_processed['subjectkey'].values)  
        y_valid_true.append(Y_valid)        
        
        # train
        clf.fit(X_train, Y_train, eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
                        eval_name=['train', 'valid'], eval_metric=['auc'],
                        max_epochs=200 , patience=30)
        
        # result
        preds_prob_val= clf.predict_proba(X_valid)[:,1]
        y_valid_pred.append(preds_prob_val)
        y_test_pred.append(clf.predict(X_test))


        preds_acc = clf.predict(X_test)
        preds_prob = clf.predict_proba(X_test)
        test_auc = roc_auc_score(y_score=preds_prob[:,1], y_true=Y_test)
        test_acc = accuracy_score(preds_acc, Y_test)

                    
        valid_res.append(clf.best_cost)
        test_auc_res.append(test_auc)
        test_acc_res.append(test_acc)
        print('[%3d/%4d] '%(i+1, fold),'Valid score: %2f'% clf.best_cost, 'Test AUC: %.3f'%test_auc, 'Test ACC: %.3f'%test_acc)
        preds_prob_res.append(preds_prob)
        clf_res.append(clf)
    
    """"print mean of valid and test"""
    print("Validation mean: %3f"%np.mean(valid_res), "Test AUC mean: %3f "%np.mean(test_auc_res), "Test ACC mean: %3f"%np.mean(test_acc_res))
    # 22sites Cross validation 
    preds_prob=[preds_prob_res[0][i]/21+preds_prob_res[1][i]/21+ preds_prob_res[2][i]/21+ preds_prob_res[3][i]/21+ preds_prob_res[4][i]/21 
                + preds_prob_res[5][i]/21+preds_prob_res[6][i]/21+ preds_prob_res[7][i]/21+ preds_prob_res[8][i]/21+ preds_prob_res[9][i]/21
                + preds_prob_res[10][i]/21+preds_prob_res[11][i]/21+ preds_prob_res[12][i]/21+ preds_prob_res[13][i]/21+ preds_prob_res[14][i]/21
                + preds_prob_res[15][i]/21+preds_prob_res[16][i]/21+ preds_prob_res[17][i]/21+ preds_prob_res[18][i]/21+ preds_prob_res[19][i]/21
                + preds_prob_res[20][i]/21
                for i in range(len(preds_prob_res[1]))]

    preds_prob = np.array(preds_prob)
    valid_result = np.mean(valid_res)
    test_auc = np.mean(test_auc_res)
    test_acc = np.mean(test_acc_res) 
    
    return test_auc,test_acc ,valid_result, clf_res, preds_prob, X_test, Y_test, y_valid_pred, y_test_pred, y_valid_subject, y_test_subject, y_valid_true



# Run function, Split data and and cross validation.

In [17]:
def run(train_data_processed, test_data_processed, fold, Num_feat, features):
    name_test = test_data_processed['subjectkey'].values
    print("-------------------------------Training Begining-------------------------------")
    n_ = [4,8,16]
    lr_ = [2e-2, 1e-2, 5e-3, 2e-3, 1e-3, 1e-4]
    w_ = [0.01, 0.001, 0.0001]
    g_ = [0.95, 0.99, 0.9]
    ss_ = [10, 20, 30]
    all_ = [n_, lr_, w_, g_, ss_]
    h_space = [s for s in itertools.product(*all_)]
    
    # Start training
    max_hy = find_bestpar(fold, train_data_processed, test_data_processed, features)
    
    # if you want to just test the code, you should use code below
    # max_hy = h_space[0]
    
    print("-------------------------------Testing Begining-------------------------------")
    test_auc,test_acc ,valid_result, clf_res, preds_prob, X_test, Y_test, y_valid_pred, y_test_pred, y_valid_subject, y_test_subject, y_valid_true= bestpar_tuning(fold, 
                                                                                        train_data_processed, 
                                                                                        test_data_processed, 
                                                                                        max_hy, 
                                                                                        features)
    
    print("-------------------------------Important Feature-------------------------------")
    import_feat=feature(Num_feat, clf_res, test_data_processed, features)
    return test_auc,test_acc ,valid_result, clf_res, preds_prob, X_test, Y_test, import_feat, name_test, y_valid_pred, y_test_pred, y_valid_subject, y_test_subject, y_valid_true


In [18]:
# for drawing ROC curve in R (extracting)

def save_prob_with_true(model):
    modeltype= "" 
    combined_model=pd.DataFrame({f"subjectkey": model.y_test_subject[0], f"Y_{modeltype}":model.Y_test, f"preds_prob_{modeltype}" :model.preds_prob[:,1]} )
    combined_model.to_csv(f"combined_forROC_{modeltype}_.csv")
    return combined_model
def save_prob_with_true_valid(model):
    modeltype= "" 
    combined_model=pd.DataFrame({f"subjectkey": list(itertools.chain(*model.y_valid_subject)), f"Y_{modeltype}":list(itertools.chain(*model.y_valid_true)), f"preds_prob_{modeltype}" :list(itertools.chain(*model.y_valid_pred) )})
    combined_model.to_csv(f"combined_forROC_{modeltype}_.csv")
    return combined_model

# Main code 

In [19]:
class model():
    def __init__(self, train_data_processed, test_data_processed, Num_FOLDS, Num_feat, features):
        test_auc,test_acc ,valid_result, clf_res, preds_prob, X_test, Y_test, import_feat, name_test, y_valid_pred, y_test_pred, y_valid_subject, y_test_subject, y_valid_true = run(train_data_processed,
                                                                              test_data_processed,
                                                                              Num_FOLDS, 
                                                                              Num_feat, 
                                                                              features)
    
    
        self.train_data_processed = train_data_processed
        self.test_auc = test_auc
        self.test_acc = test_acc
        self.valid_result = valid_result
        self.clf_res = clf_res 
        self.preds_prob = preds_prob 
        self.X_test = X_test
        self.Y_test = Y_test
        self.import_feat =  import_feat
        self.name_test = name_test
        self.features = features
        self.y_valid_pred =y_valid_pred
        self.y_test_pred = y_test_pred
        self.y_valid_subject= y_valid_subject
        self.y_test_subject = y_test_subject
        self.y_valid_true = y_valid_true
        
        test_prob_result = save_prob_with_true(self)
        valid_prob_result = save_prob_with_true_valid(self)
        self.test_prob_result= test_prob_result
        self.valid_prob_result = valid_prob_result        

In [20]:
train_data_processed, test_data_processed = preprocessing (train_data, test_data, Num_FOLDS)


done preprocessing


In [ ]:
features_mor = [col for col in train_data_processed.columns if col in mor]

mor_model = model(train_data_processed, test_data_processed, 22, Num_feat, features_mor)

In [ ]:
features_con = [col for col in train_data_processed.columns if col in con]

con_model = model(train_data_processed, test_data_processed, 22, Num_feat, features_con)

In [ ]:
features_con_mor = [col for col in train_data_processed.columns if col in con+mor]

con_mor_model = model(train_data_processed, test_data_processed, 22, Num_feat, features_con_mor)

# Model saving

In [ ]:
exp_type = "sex_classify"

with open(f'./{exp_type}/mor_model.pkl', 'wb') as f:
    dill.dump(mor_model, f)

In [ ]:
exp_type = "sex_classify"

with open(f'./{exp_type}/con_model.pkl', 'wb') as f:
    dill.dump(con_model, f)

In [ ]:
exp_type = "sex_classify"

with open(f'./{exp_type}/con_mor_model.pkl', 'wb') as f:
    dill.dump(con_mor_model, f)

# For drawing ROC curve in R (extracting)


In [ ]:
def ext_csv(filename):
    name = "mor+con"
    print(name)
    csv_test= filename.test_prob_result
    csv_valid= filename.valid_prob_result
    csv_test.to_csv(f"./CSV/{name}_test.csv")
    csv_valid.to_csv(f"./CSV/{name}_valid.csv")

In [ ]:
ext_csv(mor_model)
ext_csv(con_model)
ext_csv(con_mor_model)
